In [51]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import sympy as sp 
import numpy as np 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
# initial variables 

a = 0 
b = 1 
l = 2 # for a 4 slot problem 
h = 2 **(-l) # distance between xi, xi+1 is 1 * 2**-l 
n1 = 2**l + 1 # number of nodes 

n2 = 5 # number of roots in each GL 
ijlist = list(range(2**l-1))

dim = 2**l -1 # number of phi functions i have for l 

### sanity check ###
"""
if l = 2 i expect to have 4 slots and 3 functions and 5 points 
2**l = 2**2 = 4 --> [a = x0,x1,x2,x3,x4 = b]
dim of matrix = 3 = 2**2 -1 
gradient = 1 / 2**-l 
n1 = 5  --> number of points 
"""


x_list = gl.listi(a,b,h,n1)
print(x_list)


a = gl.a(0.25)
print(a)

matrix = sp.zeros(dim, dim)
print(matrix)






[0.0, 0.25, 0.5, 0.75, 1.0]
2.2474039592545227
Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [73]:
results = gl.gauleg(n2 = 5)
print(results)
print(results[0])


([-0.906179845938664, -0.5384693101056831, 0.0, 0.5384693101056831, 0.906179845938664], [0.236926885056189, 0.47862867049936647, 0.5688888888888889, 0.47862867049936647, 0.236926885056189])
[-0.906179845938664, -0.5384693101056831, 0.0, 0.5384693101056831, 0.906179845938664]


In [74]:
gl.GL(xi = results[0], ci = results[1], y = 0.25, x= 0, n2 = 5, func= gl.a )


0.5310875782893552

In [75]:
gl.phiij(numofnodes=5, i=0, j =1 , l = l , xlist=x_list)

-9.461277757124352

In [76]:
matrix = gl.matrix(ijlist=ijlist, matrix=matrix, n1=n1, l=l, xlist=x_list )
matrix

Matrix([
[  17.958679009754, -9.46127775712435,                 0],
[-9.46127775712435,  19.7955768453892, -10.3342990882648],
[                0, -10.3342990882648,  21.3964840963557]])

In [11]:
%timeit gl.matrix(ijlist=ijlist, matrix=matrix, n1=n1, l=l, xlist=x_list )

1.08 ms ± 26.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [77]:
F_empty = sp.zeros(dim,1)
F_empty

Matrix([
[0],
[0],
[0]])

In [78]:
def intergrand(numofnodes, i, l, xlist, f, ci, xi, n2):
    def phi_j(numofnodes, i, l, k):
        def phi_j_function(s):
            if k == i:
                return 2**l * (s - (i / 2**l))
            elif k == i + 1:
                return -(2**l * (s - ((i + 2) / 2**l)))
            else:
                return 0
        return phi_j_function

    finalsum = 0
    for k in range(numofnodes-1):
        phi_j_func = phi_j(numofnodes, i, l, k)
        x = xlist[k]
        y = xlist[k+1]
        sum = 0
        for j in range(n2):
            s = 0.5 * ((y - x) * xi[j] + (y + x))
            h = f(s) * phi_j_func(s)
            prod = ci[j] * h
            sum += prod
        onegl = (y - x) / 2 * sum
        finalsum += onegl

    return finalsum


xi = gl.gauleg(n2)[0]
ci = gl.gauleg(n2)[1]
for i in ijlist:
    Fj = intergrand(numofnodes=n1, i = i, l = l, xlist=x_list, f= gl.f, ci=ci, xi = xi, n2 = n2)
    F_empty[i,0] = Fj 

F_empty

Matrix([
[1.00129127391868],
[1.23598024222926],
[1.42649827894617]])

In [82]:
a_inv = matrix.inv()
c = a_inv * F_empty
c[1]


0.249752016058062

In [ ]:
sum = 0
for i in ijlist:
    ci_phi = ci[i] * 